In [1]:
#-----Base-----#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#-----RDKit-----#
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, PandasTools, Draw, Descriptors, MACCSkeys, rdFingerprintGenerator
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.MolStandardize import rdMolStandardize

In [2]:
class Limpeza:

    def __init__(self, dataframe: pd.DataFrame) -> None:
        '''
            Input:
                - dataframe: pd.DataFrame contendo a representação SMILES
        '''

        self.dataframe = dataframe.copy()

    def dados_limpos(
        self, 
        col_smiles: str, 
        col_valor: str, 
        sanitize: bool=True, 
        fragmento: bool=False, 
        cutoff: float=0.05,
    ) -> pd.DataFrame:
        
        '''
            Input:
                - col_smiles: nome da coluna contendo a representação SMILES
                - sanitize: se True, sanitiza as moléculas e retorna apenas as válidas.
                - fragmento: se True, avalia o fragmento principal das moléculas.
                - col_valor: nome da coluna contendo o valor a ser analisado para as métricas
                - cutoff: valor de corte para variação dos dados repetidos (0-1)
        '''

        df = self.canonical_smiles(col_smiles=col_smiles, sanitize=sanitize)
        df.dropna(axis=0, inplace=True, ignore_index=True)

        if fragmento:
            df = self.fragmento_principal(col_smiles=col_smiles)
            df.dropna(axis=0, inplace=True, ignore_index=True)

        df = self.limpa_repetidos(col_smiles=col_smiles, col_valor=col_valor, cutoff=cutoff)
        df.reset_index(drop=True, inplace=True)

        return df
    
    def canonical_smiles(self, col_smiles: str, sanitize: bool=True) -> pd.DataFrame:
        '''
            Descrição:
                Cria uma coluna de SMILES canônicos na DataFrame.
        
            Input:
                - col_smiles: nome da coluna contendo a representação SMILES
                - sanitize: se True, sanitiza as moléculas e retorna apenas as válidas.
        '''

        # Inicia uma lista vazia para receber os SMILES canônicos
        canonical_smiles = []
        valid_indices = []  # Lista para armazenar os índices válidos
        
        # Define a representação canônica de cada registro
        for index in self.dataframe.index:
            try:
                mol = Chem.MolFromSmiles(self.dataframe[col_smiles].iloc[index])
                
                # Sanitiza a molécula, se a opção estiver ativada
                if sanitize:
                    try:
                        Chem.SanitizeMol(mol)  # Sanitiza a molécula
                        mol_sanitize = rdMolStandardize.Cleanup(mol)  # Remove Hs, desconecta átomos de metal, normaliza e reioniza a molécula

                        Uncharger = rdMolStandardize.Uncharger()
                        mol_sanitize = Uncharger.uncharge(mol_sanitize)

                        Tautomer = rdMolStandardize.TautomerEnumerator()
                        tautomero = Tautomer.Canonicalize(mol_sanitize)

                        smile = Chem.MolToSmiles(tautomero, isomericSmiles=False)
                    except:
                        smile = np.nan
                else:
                    smile = Chem.MolToSmiles(mol, isomericSmiles=False)
            except Exception as e:
                smile = np.nan  # Em caso de falha, marca como NaN
            
            canonical_smiles.append(smile)
            
            # Se a molécula for válida (não NaN), adiciona o índice à lista de válidos
            if smile is not np.nan:
                valid_indices.append(index)
    
        # Cria a coluna com SMILES canônicos
        self.dataframe['smiles'] = canonical_smiles
        
        # Caso 'col_smiles' seja diferente de 'canonical_smiles', a coluna de SMILES original é removida
        if col_smiles != 'smiles':
            self.dataframe.drop([col_smiles], axis=1, inplace=True)
        
        # Se a sanitização estiver ativada, retorna apenas os registros válidos
        if sanitize:
            self.dataframe = self.dataframe.loc[valid_indices].reset_index(drop=True)
    
        return self.dataframe

    def fragmento_principal(self, col_smiles: str) -> pd.Series:
        '''
            Descrição:
                Retorna o fragmento principal (parent) da molécula.
                Ideal para clusterizações generalistas.
        
            Input:
                - col_smiles: nome da coluna contendo a representação canônica SMILES
        '''

        # Inicia uma lista vazia para receber os fragmentos
        fragmentos = []

        # Define a representação canônica de cada registro
        for index in self.dataframe.index:
            try:
                mol = Chem.MolFromSmiles(self.dataframe[col_smiles].iloc[index])
                FragmentParent = rdMolStandardize.FragmentParent(mol)
                fragmento = Chem.MolToSmiles(FragmentParent, isomericSmiles=False)

            except:
                fragmento = np.nan  # Em caso de falha, marca como NaN
            
            fragmentos.append(fragmento)
            
        # Cria a coluna com SMILES canônicos
        self.dataframe['smiles'] = fragmentos
        
        return self.dataframe

    def smiles_repetidos(self, col_smiles: str) -> pd.Series:
        '''
            Observação: Esta função foi criada para fazer parte da função limpa_repetidos().
            Não há necessidade de ser utilizada pelo usuário, exceto se o objetivo for avaliar os casos de registros repetidos.
            Para fins puramente de visualização.

            Descrição:
                Lista os registros com SMILES repetidos dentro da DataFrame.
        
            Input:
                - col_smiles: nome da coluna contendo a representação canônica SMILES
        '''
    
        # Cria uma lista com os valores SMILES repetidos na DataFrame
        smiles_duplicados = self.dataframe['smiles'][self.dataframe[col_smiles].duplicated()]
    
        # Retorna a lista de SMILES, em ordem crescente
        return smiles_duplicados.sort_values()
    
    def eda_repetidos(self, col_smiles: str, duplicados: pd.Series, col_valor: str) -> pd.DataFrame:
        '''
            Observação: Esta função foi criada para fazer parte da função limpa_repetidos().
            Não há necessidade de ser utilizada pelo usuário, exceto se o objetivo for avaliar os casos de registros repetidos.
            Para fins puramente de visualização.
            
            Descrição:
                Realiza a análise exploratória dos registros com SMILES repetidos e define a variação (%) do valor analisado.
        
            Input:
                - col_smiles: nome da coluna contendo a representação canônica SMILES
                - duplicados: lista de SMILES duplicados na DataFrame
                - col_valor: nome da coluna contendo o valor a ser analisado para as métricas
        '''
    
        # Inicia um dicionário vazio para receber a EDA associada a cada SMILES repetido
        dicio = {}
    
        # Associa o SMILES repetido à EDA
        for duplicado in duplicados:
            eda = self.dataframe[col_valor][self.dataframe[col_smiles] == duplicado].describe()
            dicio.update({duplicado : eda})
    
        # Cria a DataFrame base
        dataframe_ = pd.DataFrame(data=dicio).T

        # Calcula o Coeficiente de Variação (%) do valor dos registros com SMILES repetidos
        dataframe_['var_coef'] = (dataframe_['std'] / dataframe_['mean'])
    
        # Retorna a DataFrame da EDA de cada SMILES repetido
        return dataframe_
    
    def limpa_repetidos(self, col_smiles: str, col_valor: str, cutoff: float = 0.05) -> pd.DataFrame:
        '''
            Descrição:
                Limpa os dados repetidos com variação do valor maior que o cutoff. 
                Os valores repetidos, com pouca variação (valor <= cutoff), serão representados com a média.
        
            Input:
                - col_smiles: nome da coluna contendo a representação canônica SMILES (da 'dataframe_dados')
                - col_valor: nome da coluna contendo o valor a ser analisado para as métricas
                - cutoff: valor de corte para variação dos dados repetidos (0-1)
        '''

        # Identifica os registros repetidos
        duplicados = self.smiles_repetidos(col_smiles=col_smiles)
        # Calcula as métricas centrais para os registros repetidos
        eda = self.eda_repetidos(col_smiles=col_smiles, duplicados=duplicados, col_valor=col_valor)
        
        # Define a lista de SMILES que devem ser mantidos (ou seja, aqueles cuja variação do valor é <= ao cutoff)
        manter = eda[eda['var_coef'] <= cutoff].index
        # Remove todos os registros duplicados com variação do valor > cutoff
        dataframe_ = self.dataframe[~self.dataframe[col_smiles].isin(manter)]
    
        # Retorna a média dos registros duplicados (mas valor <= cutoff)
        return dataframe_.groupby(col_smiles, as_index=False).mean()